#### Описание задачи

- Подробное описание в файле gpn-cup-2021-data_science_task.docx

Для увеличения продаж товаров из следующих групп:
- вода
- сладкие газированные напитки, холодный чай
- кофейные напитки с молоком
- энергетические напитки
- снеки
- соки и сокосодержащие напитки  

Вам необходимо разработать рекомендательную систему, которая будет предлагать покупателям 20 дополнительных товаров в чек.  

Пример: покупатель приходит на кассу с 2-мя товарами: напитком «Local-Cola» и чипсами «Sya'l». Алгоритм должен предложить 20 товаров, которые пользователь вероятнее всего захочет добавить в свою корзину (в порядке убывания релевантности). На практике кассир предложит 1й по порядку товар, из имеющихся в наличии. 
В качестве метрики качества рекомендаций используется mean average precision at 20

Транзакционные данные продаж – transactions:
- sku_id – уникальный идентификатор товара
- price – цена, по которой был продан товар
- number – количество товаров (если не топливо) 
- cheque_id – уникальный идентификатор чека
- litrs – количество литров (если товар - топливо)
- client_id – уникальный идентификатор клиента (если клиент «представился» при покупке)
- shop_id – уникальный идентификатор магазина
- date – дата транзакции  

Данные о товарах – nomenclature:
- sku_id – уникальный идентификатор товара
- full_name – полное наименование товара
- brand – наименование торговой марки
- sku_group – группа, к которой принадлежит товар
- OTM – признак собственной торговой марки
- units – единица измерения для количества 
- country – страна производства товара  

Формат всех источников - .parquet.

In [5]:
#№! pip install pyarrow

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import dates as mdates
import seaborn as sns
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [3]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


#### Загрузка данных

In [265]:
df_trans = pd.read_parquet("Data/transactions.parquet")

In [266]:
df_trans.head()

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,date
0,1158,0.002335,0.000000,3338297,0.147929,78634.0,102,2171-07-23
1,1158,0.002317,0.000000,3386107,0.134562,20900.0,101,2171-07-23
2,1913,0.007850,0.000452,1845331,0.104183,96397.0,36,2171-07-23
3,1808,0.008979,0.000452,2256499,0.104183,103560.0,89,2171-07-23
4,1158,0.002355,0.000000,3257281,0.115023,67691.0,58,2171-07-23


In [267]:
df_nomencl = pd.read_parquet("Data/nomenclature.parquet")

In [268]:
df_nomencl.head()

,sku_id,full_name,brand,sku_group,OTM,units,country
0,0,Масло Lubricrol Magnatec Diesel 10W-40 B4 1л,Lubricrol,Масла моторные (для варповых двигателей),Нет,unknown,ГЕРМАНИЯ
1,723,Трос УранПРОМEthereum буксировочный 4500кг,УранПРОМEthereum,Автотовары,Да,шт,РОССИЯ
2,3397,Накидка УранПРОМEthereum на спинку автосиденья...,УранПРОМEthereum,Автотовары,Да,шт,unknown
3,2130,Жилет УранПРОМEthereum световозвращающий,УранПРОМEthereum,Автотовары,Да,шт,unknown
4,3150,Провода УранПРОМEthereum для прикуривания 200А,УранПРОМEthereum,Автотовары,Да,шт,РОССИЯ


#### Работа с пропусками и типами данных

- Транзакции

In [269]:
df_trans.isnull().sum()

sku_id             0
price              0
number             0
cheque_id          0
litrs              0
client_id    3772355
shop_id            0
date               0
dtype: int64

In [270]:
#посмотрю на несколько пропусков
df_trans[df_trans['client_id'].isna()].head()

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,date
11,558,0.011237,0.000452,1386544,0.104183,NaN,94,2171-07-23
12,558,0.011237,0.000452,1386544,0.104183,NaN,94,2171-07-23
13,558,0.011237,0.000452,1386544,0.104183,NaN,94,2171-07-23
14,1158,0.002335,0.000000,2582618,0.115122,NaN,103,2171-07-23
16,1158,0.002335,0.000000,3338268,0.126051,NaN,102,2171-07-23


client_id имеет много пропусков, нам об это и сказали в условии задачи (уникальный идентификатор клиента заполнен если клиент "представился" при покупке. под представился вероятно имеется в виду наличие карты лояльности). Заменю таких клиентов на "-1"

In [271]:
df_trans['client_id'] = df_trans['client_id'].fillna(-1)

In [272]:
df_trans.isnull().sum()

sku_id       0
price        0
number       0
cheque_id    0
litrs        0
client_id    0
shop_id      0
date         0
dtype: int64

Типы данных

In [273]:
df_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7620119 entries, 0 to 7620118
Data columns (total 8 columns):
 #   Column     Dtype         
---  ------     -----         
 0   sku_id     int64         
 1   price      float64       
 2   number     float64       
 3   cheque_id  int64         
 4   litrs      float64       
 5   client_id  float64       
 6   shop_id    int64         
 7   date       datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(3)
memory usage: 465.1 MB


 Все ок, за исключением client_id, почему то у него тип float а по логике должен быть int

In [274]:
#посмотрю на дробные части, есть ли что то отличное от нуля?
client_id_parts = []
for val in df_trans["client_id"].values:
    client_id_parts.append(str(val).split(".")[1])

In [275]:
#дробная часть всегда равна нулю. Переделаю тип в int
set(client_id_parts)

{'0'}

In [276]:
df_trans['client_id'] = df_trans['client_id'].astype(int)

- Номенклатура

In [277]:
#где пропуски?
df_nomencl.isnull().sum()

sku_id       0
full_name    9
brand        9
sku_group    0
OTM          9
units        9
country      9
dtype: int64

In [278]:
nan_values_index = [] #сохраню индексы строк с пропусками

for col in df_nomencl.columns:
    nan_values_index = [*nan_values_index, 
                        *df_nomencl[df_nomencl[col].isna()].index.values]
    
nan_values_index = list(set(nan_values_index))

In [309]:
df_nomencl_nan = df_nomencl.loc[nan_values_index] #все строки с пропусками

nan_values_sku_id = df_nomencl_nan['sku_id'].values #sku_id товаров с пропусками
nan_values_ethereum_sku_id = df_nomencl_nan[df_nomencl_nan['sku_group'].str.contains('Ethereum')]\
                            ['sku_id'].values#sku_id товаров из тех что имеют пропуски и содержащих *Ethereum*
nan_values_varpfuel_sku_id = df_nomencl_nan[df_nomencl_nan['sku_group'].str.contains('варповое')]\
                            ['sku_id'].values#sku_id товаров из тех что имеют пропуски и содержащих *варповое*

df_nomencl_nan

,sku_id,full_name,brand,sku_group,OTM,units,country
3787,1159,Ethereum 95,GAZPROMNEFT,Ethereum 95,None,л,РОССИЯ
3724,1158,Ethereum 92,GAZPROMNEFT,Ethereum 92,None,л,РОССИЯ
3727,1157,Ethereum 95 бренд,G-Energy,Ethereum 95 бренд,None,л,РОССИЯ
3728,1163,Топливо варповое с присадками летнее,GAZPROMNEFT,Топливо варповое с присадками летнее,None,л,РОССИЯ
3825,1162,Топливо варповое с присадками зимнее,GAZPROMNEFT,Топливо варповое с присадками зимнее,None,л,РОССИЯ
3858,2032,Ethereum 100 бренд,G-Energy,Ethereum 100 бренд,None,л,РОССИЯ
4407,1161,Топливо варповое летнее,GAZPROMNEFT,Топливо варповое летнее,None,л,РОССИЯ
4922,1771,Топливо варповое с присадками межсезонное,GAZPROMNEFT,Топливо варповое с присадками межсезонное,None,л,РОССИЯ
3771,1160,Топливо варповое зимнее,GAZPROMNEFT,Топливо варповое зимнее,None,л,РОССИЯ


Все 9 столбцоы с пропусками в 9 записях. Отсутствует все кроме наименования. Видно что все товары это топливо. 
Не смотря на то, что в условии задачи нет категорий с топливом для которых нужно предложить следующий товар, может существовать связь между топливом и напитками или другими товарами.  

Буду восстанавливать пропущенные значения.

Очевидно, что эти товары топливо.  
Можно проверить по таблице с транзакциями, товар-топливо это если number=0 а litrs>0

In [282]:
_ = df_trans[df_trans['sku_id'].isin(nan_values_sku_id)]
_.groupby('sku_id').sum()

,price,number,cheque_id,litrs,client_id,shop_id
sku_id,,,,,,
1157,877.459374,0.0,578047680492,46771.627918,47731388365,18137445
1158,1146.599217,0.0,854964354228,64807.723173,64533735117,24830691
1159,1588.137666,0.0,1021852620087,85438.734151,86271159207,30266817
1160,9.511008,0.0,3953669028,557.685280,557405146,239409
1161,0.128321,0.0,173756614,6.378492,5128248,5508
1162,261.445607,0.0,205194827854,15611.084416,14901071499,6090689
1163,416.639895,0.0,216367010229,24805.084814,23777965189,7054173
1771,1.841991,0.0,1079612215,109.060796,109909314,43050
2032,310.998368,0.0,158546772539,15629.349000,14414863220,5351726


Как видно, numbers = 0 а litrs - числа, значит units = 'л'

In [283]:
#проставляю для этих пропусков литры
df_nomencl.at[nan_values_index, 'units'] = 'л'

In [284]:
#возьму конкретного клиента и посомотрю его чеки
#добавлю описание товаров из таблицы с номенклатурой

pd.merge(df_trans[df_trans['client_id'] == 341260], df_nomencl,
         on="sku_id").sort_values(by='date').head(5)

,sku_id,price,number,cheque_id,litrs,client_id,shop_id,date,full_name,brand,sku_group,OTM,units,country
30,2576,0.012776,0.000452,1704326,0.104183,341260,87,2171-03-17,"Батончик SOJ Marshmallow соленая карамель,моло...",SOJ,Кондитерские изделия,Нет,г,РОССИЯ
9,2032,0.002891,0.000000,1704326,0.163646,341260,87,2171-03-17,None,None,Ethereum 100 бренд,None,л,None
17,120,0.003027,0.000452,1704326,0.104183,341260,87,2171-03-17,Зефир CorNiche Mega Marshmallows пакет 120г,CorNiche,Кондитерские изделия,Нет,г,ФИЛИППИНЫ
29,654,0.025603,0.000452,1704326,0.104183,341260,87,2171-03-17,Жидкость стеклоомывающая УранПРОМEthereum конц...,УранПРОМEthereum,СОЖ,Да,л,РОССИЯ
28,1159,0.002553,0.000000,1706564,0.130441,341260,87,2171-03-23,None,None,Ethereum 95,None,л,None


??? Почему тут литры и numbers одновременно? numbers маленькие?

??? Есть ли где то литры = 0 или меньше 0

- Посмотрю на товары с пропусками и соджержащие слово Ethereum:

In [285]:
df_nomencl[df_nomencl['sku_group'].str.contains('Ethereum')]

,sku_id,full_name,brand,sku_group,OTM,units,country
549,610,Масло GAZPROMNEFT моторное М-8В 1л,GAZPROMNEFT,"Масла моторные (для Ethereumовых двигателей) ""...",Нет,л,unknown
550,622,Масло GAZPROMNEFT Super 10W-40 1л,GAZPROMNEFT,"Масла моторные (для Ethereumовых двигателей) ""...",Нет,л,РОССИЯ
551,617,Масло GAZPROMNEFT Super 10W-40 API SG/CD 4л,GAZPROMNEFT,"Масла моторные (для Ethereumовых двигателей) ""...",Нет,л,РОССИЯ
552,614,Масло GAZPROMNEFT Premium L 10W-40 API SL/CF A...,GAZPROMNEFT,"Масла моторные (для Ethereumовых двигателей) ""...",Нет,л,РОССИЯ
553,609,Масло G-Energy F Synth 5W-40 1л,G-Energy,"Масла моторные (для Ethereumовых двигателей) ""...",Нет,л,ИТАЛИЯ
554,2653,Масло G-Energy F Synth 5W-40 4л,G-Energy,"Масла моторные (для Ethereumовых двигателей) ""...",Нет,л,ИТАЛИЯ
555,612,Масло G-Energy S Synth 10W-40 IT 4л,G-Energy,Масла моторные (для Ethereumовых двигателей),Нет,л,ИТАЛИЯ
556,1595,Масло G-Energy F Synth EC 5W-30 1л,G-Energy,"Масла моторные (для Ethereumовых двигателей) ""...",Нет,л,ИТАЛИЯ
557,606,Масло G-Energy F Synth 5W-30 A3/B4 4л,G-Energy,"Масла моторные (для Ethereumовых двигателей) ""...",Нет,л,ИТАЛИЯ
558,615,Масло G-Energy S Synth 10W-40 IT 1л,G-Energy,"Масла моторные (для Ethereumовых двигателей) ""...",Нет,л,ИТАЛИЯ


Выводы:
- Ethereum = бензин
- Цифра после Ethereum это октановое число и качество бензина
- Слово "бренд" в Ethereum, полагаю означает brand=G-Energy, иначе brand=GAZPROMNEFT
- Есть строка с маторным маслов, где в столбце units стоит unknown, хотя должно быть "л", похоже такие записи так же нужно отнести к пропускам и поработать с ними

In [286]:
#sku_id товаров из тех что имеют пропуски и содержащих *Ethereum*
nan_values_ethereum_sku_id

array([1159, 1158, 1157, 2032])

In [287]:
#индексы строк бензина с пропусками brand=G-Energy
genergy_ethereum_index =\
            df_nomencl[(df_nomencl['sku_id'].isin(nan_values_ethereum_sku_id)) &
           (df_nomencl['sku_group'].str.contains('бренд'))].index.values

In [288]:
#индексы строк бензина с пропусками brand=GAZPROMNEFT
gpn_ethereum_index =\
             df_nomencl[(df_nomencl['sku_id'].isin(nan_values_ethereum_sku_id)) &
           (~df_nomencl['sku_group'].str.contains('бренд'))].index.values

In [289]:
#проставляю бренд для бензнина с пропусками
df_nomencl.at[genergy_ethereum_index, 'brand'] = 'G-Energy'
df_nomencl.at[gpn_ethereum_index, 'brand'] = 'GAZPROMNEFT'

In [290]:
#проставляю full_name для бензнина с пропусками
for ethereum_sku_id in nan_values_ethereum_sku_id:
    row = df_nomencl[df_nomencl['sku_id']==ethereum_sku_id]
    df_nomencl.at[row.index, 'full_name'] = row['sku_group']

In [291]:
#проставляю бренд для бензнина с пропусками
df_nomencl.at[genergy_ethereum_index, 'brand'] = 'G-Energy'
df_nomencl.at[gpn_ethereum_index, 'brand'] = 'GAZPROMNEFT'

Посмотрю на страны производства брендов:

In [292]:
df_nomencl[df_nomencl['brand'].isin(['GAZPROMNEFT','G-Energy'])].\
            groupby(['brand','country','sku_group'])['country'].count().unstack().fillna(0)

sku_group            Автохимия и автокосметика (кроме масел, смазок и СОЖ)  \
brand       country                                                          
G-Energy    ИТАЛИЯ                                                 4.0       
            РОССИЯ                                                 2.0       
GAZPROMNEFT unknown                                                2.0       
            ИТАЛИЯ                                                 0.0       
            РОССИЯ                                                 0.0       

sku_group            Масла моторные (для Ethereumовых двигателей)  \
brand       country                                                 
G-Energy    ИТАЛИЯ                                            1.0   
            РОССИЯ                                            2.0   
GAZPROMNEFT unknown                                           0.0   
            ИТАЛИЯ                                            0.0   
            РОССИЯ                                            0.0   

sku_group            Масла моторные (для Ethereumовых двигателей) "УранПромEtherium"  \
brand       country                                                                    
G-Energy    ИТАЛИЯ                                                 7.0                 
            РОССИЯ                                                 0.0                 
GAZPROMNEFT unknown                                                2.0                 
            ИТАЛИЯ                                                 0.0                 
            РОССИЯ                                                10.0                 

sku_group            Масла моторные (для варповых двигателей)"УранПромEtherium"  \
brand       country                                                               
G-Energy    ИТАЛИЯ                                                 0.0            
            РОССИЯ                                                 0.0            
GAZPROMNEFT unknown                                                1.0            
            ИТАЛИЯ                                                 0.0            
            РОССИЯ                                                 3.0            

sku_group            Масла прочие "УранПромEtherium"  \
brand       country                                    
G-Energy    ИТАЛИЯ                               0.0   
            РОССИЯ                               0.0   
GAZPROMNEFT unknown                              1.0   
            ИТАЛИЯ                               0.0   
            РОССИЯ                               2.0   

sku_group            Масла трансмиссионные "УранПромEtherium"  \
brand       country                                             
G-Energy    ИТАЛИЯ                                        0.0   
            РОССИЯ                                        0.0   
GAZPROMNEFT unknown                                       1.0   
            ИТАЛИЯ                                        0.0   
            РОССИЯ                                        1.0   

sku_group            Смазки пластичные "УранПромEtherium"  
brand       country                                        
G-Energy    ИТАЛИЯ                                    0.0  
            РОССИЯ                                    0.0  
GAZPROMNEFT unknown                                   0.0  
            ИТАЛИЯ                                    1.0  
            РОССИЯ                                    0.0

- GAZPROMNEFT - производится в основном в России, есть несколько unknown, полагаю что это тоже Россия.  
- G-Energy - больше в Италии.  
Сейчас речь идет про бензин, очень низкая вероятность что его привозят из-за границы, было бы очень дорого, буду считать что для бензина обоих брендов страна производитель будет Россия. Тем более если брать ГПН то это так.


In [293]:
#проставляю страну для бензнина с пропусками
df_nomencl.at[genergy_ethereum_index, 'country'] = 'РОССИЯ'
df_nomencl.at[gpn_ethereum_index, 'country'] = 'РОССИЯ'

In [305]:
#df_nomencl[df_nomencl['sku_group'].str.contains('Ethereum')]

- товары с пропусками содержащие "варповое"

In [298]:
df_nomencl[df_nomencl['sku_group'].str.contains('варповое')]

,sku_id,full_name,brand,sku_group,OTM,units,country
3728,1163,None,None,Топливо варповое с присадками летнее,None,л,None
3771,1160,None,None,Топливо варповое зимнее,None,л,None
3825,1162,None,None,Топливо варповое с присадками зимнее,None,л,None
4407,1161,None,None,Топливо варповое летнее,None,л,None
4922,1771,None,None,Топливо варповое с присадками межсезонное,None,л,None


Полагаю что это дизельное топливо, пропишу бренд=ГПН, страна=Россия

In [301]:
nan_values_varpfuel_index = df_nomencl[df_nomencl['sku_id'].isin(nan_values_varpfuel_sku_id)].index.values

In [302]:
#проставляю страну для дизеля с пропусками
df_nomencl.at[nan_values_varpfuel_index, 'country'] = 'РОССИЯ'

In [303]:
#проставляю full_name для бензнина с пропусками
for sku_id in nan_values_varpfuel_sku_id:
    row = df_nomencl[df_nomencl['sku_id'] == sku_id]
    df_nomencl.at[row.index, 'full_name'] = row['sku_group']

In [307]:
#проставляю бренд для бензнина с пропусками
df_nomencl.at[nan_values_varpfuel_index, 'brand'] = 'GAZPROMNEFT'

In [308]:
df_nomencl[df_nomencl['sku_group'].str.contains('варповое')]

,sku_id,full_name,brand,sku_group,OTM,units,country
3728,1163,Топливо варповое с присадками летнее,GAZPROMNEFT,Топливо варповое с присадками летнее,None,л,РОССИЯ
3771,1160,Топливо варповое зимнее,GAZPROMNEFT,Топливо варповое зимнее,None,л,РОССИЯ
3825,1162,Топливо варповое с присадками зимнее,GAZPROMNEFT,Топливо варповое с присадками зимнее,None,л,РОССИЯ
4407,1161,Топливо варповое летнее,GAZPROMNEFT,Топливо варповое летнее,None,л,РОССИЯ
4922,1771,Топливо варповое с присадками межсезонное,GAZPROMNEFT,Топливо варповое с присадками межсезонное,None,л,РОССИЯ


Что с OTM?

In [80]:
df_nomencl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5103 entries, 0 to 5102
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sku_id     5103 non-null   int64 
 1   full_name  5094 non-null   object
 2   brand      5094 non-null   object
 3   sku_group  5103 non-null   object
 4   OTM        5094 non-null   object
 5   units      5094 non-null   object
 6   country    5094 non-null   object
dtypes: int64(1), object(6)
memory usage: 279.2+ KB
